# Project: Decoding Molecules From Fingerprints.
## Group Members:
### Qi Chen, e-mail: gusqichr@student.gu.se
### Nils Dunlop, e-mail: gusdunlni@student.gu.se
### Francisco Alejandro Erazo Piza, e-mail: guserafr@student.gu.se
***

In [1]:
import os
import pandas as pd
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed
from rdkit import Chem

### Define directories
***

In [2]:
# Set user directory
USER_DIR = os.path.expanduser('~')

# Set the project root directory
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Define the input and output directories
INPUT_DIR = os.path.join(PROJECT_ROOT, 'data/chunks')
OUTPUT_DIR = os.path.join(PROJECT_ROOT, 'data/processed_chunks')
FINAL_OUTPUT = os.path.join(PROJECT_ROOT, 'data/final_processed_data.parquet')

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Path to the conda environment to MolForge
CONDA_ENV_PATH = os.path.join(USER_DIR, 'conda/envs/molforge/bin/python')
MOLFORGE_DIR = os.path.join(USER_DIR, 'MolForge')
MOLFORGE_SCRIPT_PATH = os.path.join(USER_DIR, 'MolForge/predict.py')

# Define the number of workers
WORKERS = 16

In [3]:
print(PROJECT_ROOT)
print(INPUT_DIR)
print(OUTPUT_DIR)
print(FINAL_OUTPUT)
print(CONDA_ENV_PATH)
print(MOLFORGE_DIR)
print(MOLFORGE_SCRIPT_PATH)

/home/student/Fingerprints-To-Smiles
/home/student/Fingerprints-To-Smiles/data/chunks
/home/student/Fingerprints-To-Smiles/data/processed_chunks
/home/student/Fingerprints-To-Smiles/data/final_processed_data.parquet
/home/student/conda/envs/molforge/bin/python
/home/student/MolForge
/home/student/MolForge/predict.py


### Check parquet files
***

In [4]:

for file in os.listdir(INPUT_DIR):
    if file.endswith('.parquet'):
        df = pd.read_parquet(os.path.join(INPUT_DIR, file))
        print(f"{file} shape: {df.shape}")

molecule_fingerprints_part_16.parquet shape: (50000, 4)
molecule_fingerprints_part_21.parquet shape: (50000, 4)
molecule_fingerprints_part_25.parquet shape: (50000, 4)
molecule_fingerprints_part_27.parquet shape: (32680, 4)
molecule_fingerprints_part_11.parquet shape: (50000, 4)
molecule_fingerprints_part_8.parquet shape: (50000, 4)
molecule_fingerprints_part_13.parquet shape: (50000, 4)
molecule_fingerprints_part_0.parquet shape: (50000, 4)
molecule_fingerprints_part_15.parquet shape: (50000, 4)
molecule_fingerprints_part_18.parquet shape: (50000, 4)
molecule_fingerprints_part_9.parquet shape: (50000, 4)
molecule_fingerprints_part_6.parquet shape: (50000, 4)
molecule_fingerprints_part_26.parquet shape: (50000, 4)
molecule_fingerprints_part_24.parquet shape: (50000, 4)
molecule_fingerprints_part_20.parquet shape: (50000, 4)
molecule_fingerprints_part_10.parquet shape: (50000, 4)
molecule_fingerprints_part_23.parquet shape: (50000, 4)
molecule_fingerprints_part_22.parquet shape: (50000,

In [5]:
parquet_files = ['molecule_fingerprints_part_0.parquet', 
                 'molecule_fingerprints_part_1.parquet', 
                 'molecule_fingerprints_part_2.parquet'
                 ]

#### Compute MolForge Predictions
***


In [6]:
def convert_spaced_smiles(spaced_smiles):
    compact_smiles = spaced_smiles.replace(" ", "")
    mol = Chem.MolFromSmiles(compact_smiles)
    if mol is not None:
        return Chem.MolToSmiles(mol, isomericSmiles=False)
    else:
        return "Invalid SMILES string"

In [7]:
def run_molforge(fp_input):
    # Construct the command
    command = [
        CONDA_ENV_PATH,
        MOLFORGE_SCRIPT_PATH,
        "--fp=ECFP4",
        "--model_type=smiles",
        f"--input={fp_input}"
    ]
    
    # Run the command and capture output
    try:
        result = subprocess.run(
            command,
            capture_output=True,
            text=True,
            check=True,
            cwd=MOLFORGE_DIR,
            env=os.environ.copy()
        )
    except subprocess.CalledProcessError as e:
        print(f"Error running MolForge: {e}")
        print(f"Error output: {e.stderr}")
        return "Error", "Error"
    
    # Process the result
    spaced_smiles = None
    device_used = None
    output_lines = result.stdout.splitlines()
    
    for line in output_lines:
        if "Result:" in line:
            spaced_smiles = line.split("Result:")[1].strip()
        if "rank :" in line:
            device_used = line.split("rank :")[1].strip()
    
    # Convert the spaced SMILES to compact SMILES
    if spaced_smiles:
        compact_smiles = convert_spaced_smiles(spaced_smiles)
    else:
        compact_smiles = "No Result"

    return compact_smiles, device_used

In [8]:
def process_row(row):
    fp_input = row['SparseFingerprintBits']
    
    # Run MolForge and get the converted SMILES and device information
    compact_smiles, device_used = run_molforge(fp_input)
    
    return compact_smiles, device_used

In [9]:
def process_chunk(file_path, max_rows=None):
    chunk_df = pd.read_parquet(file_path)
    if max_rows:
        chunk_df = chunk_df.head(max_rows)
    
    total = len(chunk_df)
    with ThreadPoolExecutor(max_workers=WORKERS) as executor:
        futures = {executor.submit(run_molforge, row['SparseFingerprintBits']): idx for idx, row in chunk_df.iterrows()}
        
        for i, future in enumerate(as_completed(futures)):
            idx = futures[future]
            result, device_used = future.result()
            chunk_df.at[idx, 'Converted_SMILES'] = result
            chunk_df.at[idx, 'Device_Used'] = device_used
            if (i + 1) % 100 == 0 or (i + 1) == total:
                print(f"Processed molecule {i + 1}/{total} in {os.path.basename(file_path)}")
    
    output_file = os.path.join(OUTPUT_DIR, f"processed_{os.path.basename(file_path)}")
    chunk_df.to_parquet(output_file, index=False)
    print(f"Finished processing {os.path.basename(file_path)}")

In [10]:
def process_files(file_list=None, max_rows=None):
    if file_list:
        chunk_files = [f for f in file_list if f in os.listdir(INPUT_DIR) and f.endswith('.parquet')]
    else:
        chunk_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.parquet')]
    
    total_chunks = len(chunk_files)
    for i, file in enumerate(chunk_files, 1):
        file_path = os.path.join(INPUT_DIR, file)
        process_chunk(file_path, max_rows)


In [11]:
process_files(file_list=parquet_files)

[22:01:08] SMILES Parse Error: unclosed ring for input: 'CN1C(CC2=C3C14CCN(C(C3(CC2)O)CC(=O)NC5=CC=CC=C5)CC6CC6)CC(=O)NC7=CC=CC=C7'


Processed molecule 100/50000 in molecule_fingerprints_part_0.parquet
Processed molecule 200/50000 in molecule_fingerprints_part_0.parquet


[22:01:59] SMILES Parse Error: unclosed ring for input: 'C1CCC2=NC3=C(C=C(C=C3)NC(=O)CCCCCCCCCCCCCCCCCCCCCCCCCC2)C(=C2C1)N'


Processed molecule 300/50000 in molecule_fingerprints_part_0.parquet
Processed molecule 400/50000 in molecule_fingerprints_part_0.parquet
Processed molecule 500/50000 in molecule_fingerprints_part_0.parquet
Processed molecule 600/50000 in molecule_fingerprints_part_0.parquet
Processed molecule 700/50000 in molecule_fingerprints_part_0.parquet


[22:04:08] SMILES Parse Error: unclosed ring for input: 'C1C2CC3CC1CC(C2)(C3)C4=CC=CC=C4C5C(C6=CC=CC=C6C57CC8CC(C7)CC(C8)C7)N'


...

Processed molecule 49000/50000 in molecule_fingerprints_part_2.parquet
Processed molecule 49100/50000 in molecule_fingerprints_part_2.parquet
Processed molecule 49200/50000 in molecule_fingerprints_part_2.parquet


[09:55:41] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 6 7 8 9 10
[09:55:43] SMILES Parse Error: unclosed ring for input: 'COC(=O)C1(CC23CCCN4C2(CCC4)C5=C(C1(C#N)O)C6=C(C=C5)OCO6)O'


Processed molecule 49300/50000 in molecule_fingerprints_part_2.parquet
Processed molecule 49400/50000 in molecule_fingerprints_part_2.parquet
Processed molecule 49500/50000 in molecule_fingerprints_part_2.parquet
Processed molecule 49600/50000 in molecule_fingerprints_part_2.parquet


[09:57:54] SMILES Parse Error: unclosed ring for input: 'C/C=C\1/CN2[C@H]3C[C@@H]1[C@@]([C@@H]2C[C@@]45[C@H]3N(C6=CC=CC=C64)C)(C(=O)OC)OC(=O)C7=CC(=C(C=C7)OC)OC'
[09:58:11] SMILES Parse Error: unclosed ring for input: 'C[C@@H]1[C@@H]2[C@@H]3C[C@@H]4C5=C(CCN4C(=O)[C@H]2N(O1)C)C6=CC=CC=C6N5'


Processed molecule 49700/50000 in molecule_fingerprints_part_2.parquet
Processed molecule 49800/50000 in molecule_fingerprints_part_2.parquet
Processed molecule 49900/50000 in molecule_fingerprints_part_2.parquet
Processed molecule 50000/50000 in molecule_fingerprints_part_2.parquet
Finished processing molecule_fingerprints_part_2.parquet
